In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from preprocessing import *
from functools import partial
from typing import Iterable, Any
from itertools import product
from time import time
import numpy as np
import pandas as pd
import random
import zipfile
from glob import glob
import os
from sklearn.model_selection import ParameterGrid

2022-12-07 14:25:40.964472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 14:25:41.070870: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-07 14:25:41.075727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 14:25:41.075745: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
PREPROCESSING_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
    'num_coefficients':40
}

TRAINING_ARGS = {
    'batch_size': 20,
    'epochs': 10,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
}

final_sparsity = 0.70

In [ ]:
train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])
go_stop_label = LABELS

def get_mfcc_and_label(filename, downsampling_rate, frame_length_in_s, frame_step_in_s, num_mel_bins, lower_frequency, upper_frequency, num_coefficients):
    mfccs, label = get_mfccs(filename, downsampling_rate, frame_length_in_s, frame_step_in_s, num_mel_bins, lower_frequency, upper_frequency, num_coefficients)
    
    return mfccs, label

get_frozen_spectrogram = partial(get_mfcc_and_label, **PREPROCESSING_ARGS)

for spectrogram, label in train_ds.map(get_frozen_spectrogram).take(1):
    SHAPE = spectrogram.shape
SHAPE



2022-12-07 14:25:47.269431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-07 14:25:47.269466: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-07 14:25:47.269483: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-d234ae94-1d7d-4b6d-92da-bcf2d3a86b11): /proc/driver/nvidia/version does not exist
2022-12-07 14:25:47.269752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 14:25:47.768363: W tensorflow_io/core/kernels/audio_v

TensorShape([49, 40])

In [ ]:
def preprocess(filename):
    signal, label = get_frozen_spectrogram(filename)

    signal.set_shape(SHAPE)
    signal = tf.expand_dims(signal, -1)
    signal = tf.image.resize(signal, [32, 32])

    label_id = tf.argmax(label == LABELS)

    return signal, label_id

batch_size = TRAINING_ARGS['batch_size']
epoch = TRAINING_ARGS['epochs']

train_ds = train_ds.map(preprocess).batch(batch_size).cache()
val_ds = val_ds.map(preprocess).batch(batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size)

for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

2022-12-07 14:25:48.910006: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:48.911845: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:48.912033: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:49.278474: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:49.280152: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:49.280333: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-07 14:25:49.631760: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed 

In [ ]:
model_DS = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], 
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[1, 1], strides=[1, 1],   
       use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
    ])
  
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

begin_step = int(len(train_ds) * epoch * 0.2)
end_step = int(len(train_ds) * epoch)

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
    }

model_for_pruning = prune_low_magnitude(model_DS, **pruning_params)


In [ ]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 15, 15, 128)      2306      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 15, 15, 128)      513       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 15, 15, 128)      1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_depthwi  (None, 15, 15, 128)      1153      
 se_conv2d (PruneLowMagnitud                                     
 e)                                                     

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epoch,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epoch, validation_data=val_ds, callbacks=callbacks)
for layer in model_for_pruning.layers:
    if isinstance(layer, tf.keras.layers.Wrapper):
        weights = layer.trainable_weights
    else:
        weights = layer.weights

test_loss, test_accuracy =  model_for_pruning.evaluate(test_ds)

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training loss: {training_loss:.3f}')
print(f'Training accuracy: {training_accuracy * 100:.2f}%')
print(f'Validation loss: {val_loss:.3f}')
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.3f}')
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/10
80/80 [==============================] - 26s 280ms/step - loss: 0.5236 - sparse_categorical_accuracy: 0.7487 - val_loss: 0.3835 - val_sparse_categorical_accuracy: 0.8300
Epoch 2/10
80/80 [==============================] - 6s 75ms/step - loss: 0.3807 - sparse_categorical_accuracy: 0.8394 - val_loss: 0.3537 - val_sparse_categorical_accuracy: 0.8950
Epoch 3/10
80/80 [==============================] - 6s 73ms/step - loss: 0.2860 - sparse_categorical_accuracy: 0.8869 - val_loss: 0.6954 - val_sparse_categorical_accuracy: 0.7100
Epoch 4/10
80/80 [==============================] - 6s 72ms/step - loss: 0.2284 - sparse_categorical_accuracy: 0.9087 - val_loss: 0.2591 - val_sparse_categorical_accuracy: 0.9200
Epoch 5/10
80/80 [==============================] - 6s 74ms/step - loss: 0.1743 - sparse_categorical_accuracy: 0.9294 - val_loss: 0.4725 - val_sparse_categorical_accuracy: 0.8050
Epoch 6/10
80/80 [==============================] - 6s 72ms/step - loss: 0.1538 - sparse_categorical_ac

In [ ]:
from time import time
#SAVE THE MODEL
timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model_for_pruning.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1670423423/assets
INFO:tensorflow:Assets written to: ./saved_models/1670423423/assets


In [ ]:
MODEL_NAME=saved_model_dir.split('/')[2]
#CONVERT THE MODEL TO TFLITE
converter = tf.lite.TFLiteConverter.from_saved_model(f'{saved_model_dir}')
tflite_model = converter.convert()
tflite_models_dir = './tflite_models'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

2022-12-07 14:33:53.886423: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-07 14:33:53.886467: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-07 14:33:53.886602: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1670423423
2022-12-07 14:33:53.897277: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-07 14:33:53.897317: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/1670423423
2022-12-07 14:33:53.936045: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-07 14:33:54.044942: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./saved_models/1670423423
2022-12-07 14:33:54.083076: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status

In [ ]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')
tflite_model_name

'./tflite_models/1670423423.tflite'

In [ ]:
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)
    
import zipfile
#COMPRESS THE MODEL
with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

tflite_size = os.path.getsize(tflite_model_name) / 1024.0  
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size (pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')


2022-12-07 14:27:19.132871: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-07 14:27:19.132923: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-07 14:27:19.133753: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/1670423232
2022-12-07 14:27:19.144544: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-07 14:27:19.144580: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/1670423232
2022-12-07 14:27:19.190154: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-07 14:27:19.197916: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-07 14:27:19.311861: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./save

FileNotFoundError: [Errno 2] No such file or directory: './tflite_models/1670423232.tflite'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>